# Homework 1 - Training a simple model
Based on the Notebook shared on the courses material and related to the optional video about training a simple model.
In the end I basically cleaned up, reorganized, pointed to new data sources and tried to play around a bit.
[Homework URL](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md) 

## Part 0 - Technicalities
Setting up the environment.

Lets execute in the console directly with ! some installing. This was needd as conda was not working very well for me and I just made a python .venv 

_Note: I had a lot of technical troubles, with the jupyter notebook crashing all the time!_

In [ ]:
!python -V

Libraries needed here

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install pandas
!pip install seaborn
!pip install scikit-learn

Pylance includes a parquet file reader for our datasources.

In [ ]:
!pip install pylance

Getting those libraries.
At some point I just commented some of those imports, when not using them for some tests, to speed up execution. 

In [ ]:
import pandas as pd
#import pickle
#import seaborn as sns
#import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import Lasso
#from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

## Part 1 - First exploration and playing around

Import data for Yellow NYC Taxis Jan 2023 in parquet format

In [ ]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

Some information cells to execute independently (to know how many columns, their distribution, averages, etc)

In [ ]:
df.info()

In [ ]:
df.describe()

Creation of a new column for trip duration

In [ ]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df['duration'].describe()

df_short to compare with original df and check how much is dropped if we filter for trips between 1 minute and 1 hour (60min)

In [ ]:
df_short = df[(df.duration >= 1) & (df.duration <= 60)]

df_short['duration'].count()
df_short['duration'].describe()
df_short['duration'].count()/df['duration'].count()

From [homework](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md) 

_Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model._

-_Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)_

-_Fit a dictionary vectorizer_

-_Get a feature matrix from it_

In [ ]:
df = df_short #lets now take the short as our main working set

df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID'] #one hot encoding
numerical = ['trip_distance']
df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

#We're using a simple linear regression as requested on homework
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

### Optional Plotting

In [ ]:
sns.histplot(y_pred, label='prediction')
sns.histplot(y_train, label='actual')

plt.legend()

## Part 2 - Main Part

Creating the import function and cleaning, so that we can have some easy repeatability of what we've done previously

In [ ]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)
    categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

### Importing Data
_Separated into different cells to control and test execution, as the kernel was constantly crashing!_

In [ ]:
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')

In [ ]:
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

Doublecheck sizes

In [ ]:
len(df_train), len(df_val)

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']  
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:
dv = DictVectorizer()

categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
df_train[categorical] = df_train[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)

numerical = ['trip_distance']

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

In [ ]:
lr = LinearRegression()
lr.fit(X_val, y_val)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

Pickle doesn't seem to work well in jupyter, just let it here for reference

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

Other alternatives that were left here for reference (lasso algorithm)

In [ ]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)